In [1]:
#!pip install -r requirements.txt

In [2]:
import yaml
import timeit
import numpy as np
import pandas as pd
import torch
from datetime import datetime
from utility import *
from ultralytics import settings
from registry import get_model, get_pipeline
#settings.update({'datasets_dir': os.path.abspath('.')})


Install mish-cuda to speed up training and inference. More importantly, replace the naive Mish with MishCuda will give a ~1.5G memory saving during training.
c:\Users\user\anaconda3\envs\AerialObjectDetection\lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
c:\Users\user\anaconda3\envs\AerialObjectDetection\lib\site-packages\timm\models\registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
C:\Users\user\Desktop\Aerial Object Detection\AERIAL_OBJECT_DETECTION\Models\YOLOH\models\backbone\convnext.py:204: UserWarning: Overwriting convnext_tiny in registry with models.backbone.convnext.convnext_tiny. This is because the name bei

In [3]:
project_name = 'Object Detection'
# dataset_root 가 상대경로일 경우 colab 환경에서는 무조건 앞에 datasets 를 붙여서 찾는 문제가 있음
dataset_root = 'Datasets'
output_dir = 'output'
dataset_names = ['NWPU_VHR10_YOLO', 'COCO_airplane',]
model_names = ['yolov8n']
iterations = [1, 5]
epochs = 10
early_stop = 20
batch_size = 16
image_size = 640
optimizer = 'AdamW'
lr = 0.001
momentum = 0.9
weight_decay = 1e-4
optim_args = {'optimizer': optimizer, 'lr': lr, 'momentum': momentum, 'weight_decay': weight_decay}
devices = [0]
device = torch.device("cuda:"+str(devices[0])) if len(devices)>0 else torch.device("cpu")
train_split = 0.6
val_split = 0.2
test_split = 0.2
num_workers = 0

In [4]:
from utility.utils import control_random_seed, update_dataset_paths, format_measures, merge_and_update_df


Experiments_Time = datetime.now().strftime("%y%m%d_%H%M%S")
print('Experiment Start Time:',Experiments_Time)

ex_dict = {}
ex_dict['Experiment Time'] = Experiments_Time;ex_dict['Epochs'] = epochs;
ex_dict['Batch Size'] = batch_size;
ex_dict['Early Stop'] = early_stop;
ex_dict['Device'] = device
ex_dict['Optimizer'] = optimizer;
ex_dict['LR']=optim_args['lr']; ex_dict['Weight Decay']=optim_args['weight_decay'];ex_dict['Momentum']=optim_args['momentum'];
#ex_dict['Hyp'] = dict(lr0=ex_dict['LR'], momentum=ex_dict['Momentum'], weight_decay=ex_dict['Weight Decay'])
ex_dict['Image Size'] = image_size
ex_dict['Output Dir'] = output_dir
ex_dict['Num Workers'] = num_workers
ex_dict['AutoAnchor'] = True

for iteration in range(iterations[0], iterations[1]+1):
    print(f'(Iter {iteration})')
    seed = iteration
    ex_dict['Iteration'] = iteration
    for j, Dataset_Name in enumerate(dataset_names):
        print(f'Dataset: {Dataset_Name} ({j+1}/{len(dataset_names)})');
        control_random_seed(seed)
        data_yaml_path = f"{dataset_root}/{Dataset_Name}/data_iter_{iteration:02d}.yaml"
        with open(data_yaml_path, 'r') as f:
            data_config = yaml.load(f, Loader=yaml.FullLoader)
        ex_dict['Dataset Name'] = Dataset_Name; ex_dict['Data Config'] = data_yaml_path; ex_dict['Number of Classes'] = data_config['nc']; ex_dict['Class Names'] = data_config['names'];
        ex_dict['Split Size'] = f"{data_config['train_count']}/{data_config['val_count']}/{data_config['test_count']}"
        update_dataset_paths(dataset_root, Dataset_Name, iteration)
        
        for k, model_name in enumerate(model_names):
            print(f'{model_name} ({k+1}/{len(model_names)}) (Iter {iteration})', end=' ')
            print(f'Dataset: {Dataset_Name} ({j+1}/{len(dataset_names)})', end=' ')
            control_random_seed(seed)
            
            model = get_model(model_name, ex_dict)
            train_fn, eval_fn, test_fn = get_pipeline(model_name)

            ex_dict['Model Name'] = model_name; ex_dict['Model']=model;
            start = timeit.default_timer()

            ex_dict = train_fn(ex_dict)
            ex_dict = eval_fn(ex_dict)
            ex_dict = test_fn(ex_dict)

            ex_dict['Train-Test Time'] = timeit.default_timer() - start
            #################
            if ex_dict.get('Test Results') is None:
                raise RuntimeError("평가 결과가 없습니다. 위의 체크리스트를 참고해 원인부터 해결하세요.")

            eval_dict = format_measures(ex_dict.get('Test Results'))
            output_csv = f"{ex_dict['Experiment Time']}_{project_name}_Results.csv"
            merge_and_update_df(ex_dict, eval_dict, output_csv, exclude_columns=['Model', 'Train Results', 'Test Results'])

Experiment Start Time: 250522_154811
(Iter 1)
Dataset: NWPU_VHR10_YOLO (1/2)
yolov8n (1/4) (Iter 1) Dataset: NWPU_VHR10_YOLO (1/2) New https://pypi.org/project/ultralytics/8.3.141 available  Update with 'pip install -U ultralytics'
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=Datasets/NWPU_VHR10_YOLO/data_iter_01.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=output, name=yolov8_train11, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms

train: Scanning Datasets\NWPU_VHR10_YOLO\labels... 390 images, 0 backgrounds, 0 corrupt: 100%|██████████| 390/390 [00:00<00:00, 1657.84it/s]


train: New cache created: Datasets\NWPU_VHR10_YOLO\labels.cache


val: Scanning Datasets\NWPU_VHR10_YOLO\labels... 130 images, 0 backgrounds, 0 corrupt: 100%|██████████| 130/130 [00:00<00:00, 1373.54it/s]

val: New cache created: Datasets\NWPU_VHR10_YOLO\labels.cache


...


[YoloOW] ✅ Epoch 10/10 complete in 24.58s | Avg Loss: 6.5765

[YoloOW] 📊 Running evaluation after epoch 10 ...
[DEBUG] Head 0 | iou.min=0.1606, iou.max=0.9059, iou.mean=0.5192
[DEBUG] Head 0 | ps[..., 4].sigmoid().mean = 0.0016
[DEBUG] Head 1 | iou.min=0.1374, iou.max=0.9197, iou.mean=0.4449
[DEBUG] Head 1 | ps[..., 4].sigmoid().mean = 0.0045
[DEBUG] Head 2 | iou.min=-0.0075, iou.max=0.7542, iou.mean=0.3334
[DEBUG] Head 2 | ps[..., 4].sigmoid().mean = 0.0099
[DEBUG] Head 0 | iou.min=0.1591, iou.max=0.8519, iou.mean=0.5110
[DEBUG] Head 0 | ps[..., 4].sigmoid().mean = 0.0014
[DEBUG] Head 1 | iou.min=0.0538, iou.max=0.8602, iou.mean=0.4277
[DEBUG] Head 1 | ps[..., 4].sigmoid().mean = 0.0045
[DEBUG] Head 2 | iou.min=-0.0173, iou.max=0.8390, iou.mean=0.3150
[DEBUG] Head 2 | ps[..., 4].sigmoid().mean = 0.0100
[DEBUG] Head 0 | iou.min=0.1970, iou.max=0.8446, iou.mean=0.5016
[DEBUG] Head 0 | ps[..., 4].sigmoid().mean = 0.0014
[DEBUG] Head 1 | iou.min=0.0534, iou.max=0.9107, iou.mean=0.4185
[D

[YOLOH] Epoch 1/10:   0%|          | 0/24 [00:00<?, ?it/s]


[DEBUG] first batch target rows = 120
[DEBUG] first 5 targets:
 tensor([[0.00000, 8.00000, 0.09659, 0.54352, 0.18056, 0.28064],
        [0.00000, 8.00000, 0.25316, 0.38632, 0.17550, 0.12966],
        [0.00000, 8.00000, 0.29040, 0.26288, 0.11869, 0.19893],
        [1.00000, 5.00000, 0.37340, 0.57941, 0.11060, 0.12216],
        [1.00000, 5.00000, 0.39351, 0.68586, 0.11792, 0.15009]])
[Before Interpolate] x2: torch.Size([16, 512, 20, 20]), x1: torch.Size([16, 512, 40, 40]), x0: torch.Size([16, 512, 40, 40])
[After Interpolate]  x2: torch.Size([16, 512, 40, 40]), x1: torch.Size([16, 512, 40, 40]), x0: torch.Size([16, 512, 40, 40])


c:\Users\user\anaconda3\envs\AerialObjectDetection\lib\site-packages\torch\functional.py:554: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\native\TensorShape.cpp:4316.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
[YOLOH] Epoch 1/10:   0%|          | 0/24 [00:00<?, ?it/s]


TypeError: unsupported operand type(s) for &: 'Tensor' and 'NoneType'

In [ ]:
import torch

print(f"CUDA 사용 가능 여부: {torch.cuda.is_available()}")
print(f"CUDA 디바이스 수: {torch.cuda.device_count()}")
print(f"현재 사용 디바이스: {torch.cuda.current_device()}")
print(f"디바이스 이름: {torch.cuda.get_device_name(0)}")
print(f"PyTorch CUDA 버전: {torch.version.cuda}")
print(f"PyTorch 버전: {torch.__version__}")


CUDA 사용 가능 여부: True
CUDA 디바이스 수: 1
현재 사용 디바이스: 0
디바이스 이름: NVIDIA GeForce RTX 5060 Ti
PyTorch CUDA 버전: 12.8
PyTorch 버전: 2.8.0.dev20250420+cu128
